In [1]:
import requests
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy
import csv
import os
import time
from decimal import Decimal
from sqlalchemy.orm import Session, session
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
password = 1234

In [2]:
states = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','District of Columbia','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']
states_abbrev = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
df_columns = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','District of Columbia','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']

In [3]:
url = "https://api.census.gov/data/2019/acs/flows?get=MOVEDIN,STATE1_NAME,STATE2_NAME,MOVEDOUT,FULL2_NAME,MOVEDNET&for=county:*"

In [4]:
migration_data_2019 = pd.read_json(url)

In [5]:
#migration_data_2019

In [5]:
migration_data_2019.columns = migration_data_2019.iloc[0]
migration_data_2019 = migration_data_2019.drop(index=0)
migration_data_2019

,MOVEDIN,STATE1_NAME,STATE2_NAME,MOVEDOUT,FULL2_NAME,MOVEDNET,state,county
1,66,Alabama,Asia,None,Asia,None,01,001
2,10,Alabama,Europe,None,Europe,None,01,001
3,18,Alabama,U.S. Island Areas,None,U.S. Island Areas,None,01,001
4,25,Alabama,Alabama,448,"Baldwin County, Alabama",-423,01,001
5,4,Alabama,Alabama,0,"Barbour County, Alabama",4,01,001
...,...,...,...,...,...,...,...,...
505932,0,Puerto Rico,New York,69,"Rochester city, Monroe County, New York",-69,72,153
505933,40,Puerto Rico,New York,0,"Manhattan borough, New York County, New York",40,72,153
505934,0,Puerto Rico,Pennsylvania,14,"Upper Macungie township, Lehigh County, Pennsy...",-14,72,153
505935,0,Puerto Rico,Pennsylvania,42,"Hanover borough, York County, Pennsylvania",-42,72,153


In [6]:
migration_data_2019.columns  = [i.lower() for i in migration_data_2019.columns]

In [7]:
migration_data_2019

,movedin,state1_name,state2_name,movedout,full2_name,movednet,state,county
1,66,Alabama,Asia,None,Asia,None,01,001
2,10,Alabama,Europe,None,Europe,None,01,001
3,18,Alabama,U.S. Island Areas,None,U.S. Island Areas,None,01,001
4,25,Alabama,Alabama,448,"Baldwin County, Alabama",-423,01,001
5,4,Alabama,Alabama,0,"Barbour County, Alabama",4,01,001
...,...,...,...,...,...,...,...,...
505932,0,Puerto Rico,New York,69,"Rochester city, Monroe County, New York",-69,72,153
505933,40,Puerto Rico,New York,0,"Manhattan borough, New York County, New York",40,72,153
505934,0,Puerto Rico,Pennsylvania,14,"Upper Macungie township, Lehigh County, Pennsy...",-14,72,153
505935,0,Puerto Rico,Pennsylvania,42,"Hanover borough, York County, Pennsylvania",-42,72,153


In [8]:
not_states = ["Asia", "Europe", "U.S. Island Areas", "Africa", "Caribbean", "Oceania and At Sea", "Central America","Puerto Rico", "South America", "Northern America"] 
for i in not_states:
    migration_data_2019 = migration_data_2019[migration_data_2019["state2_name"] != i]

In [9]:
migration_data_2019["movedin"] = pd.to_numeric(migration_data_2019["movedin"])
migration_data_2019["movedout"] = pd.to_numeric(migration_data_2019["movedout"])
migration_data_2019["movednet"] = pd.to_numeric(migration_data_2019["movednet"])

In [10]:
connection_string = f"postgres:{password}@localhost:5432/project_3"
engine = create_engine(f'postgresql://{connection_string}')
if not database_exists(engine.url):
    create_database(engine.url)

In [11]:
migration_data_2019.to_sql(name='migration_data_2019', con=engine, if_exists='replace', index=False)

In [12]:
session = Session(engine)
time.sleep(2)

In [13]:
all_data = []
for i in range(len(states)):
    data = session.execute(f"select sum(movedin) from migration_data_2019 where state1_name = '{states[i]}' group by state1_name, state2_name order by state2_name").all()
    data_fixed = []
    for row in data:
        data_fixed.append(list(map(int, list(row))))
    data_fixed = [data_fixed[i][0] for i in range(len(data_fixed))]
    #data_fixed[i] = "NaN"
    all_data.append(data_fixed)

In [87]:
#[data_fixed[i][0] for i in range(len(data_fixed))]

In [14]:
df = pd.DataFrame(data=all_data, index=[states_abbrev, states], columns=df_columns)

In [15]:
for i in range(len(states)):
    df.iloc[i,i] = "N/A"

In [16]:
df

,,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
AL,Alabama,N/A,658,1496,897,3760,1984,1354,228,229,15510,...,84,9062,7611,748,72,3663,1315,396,1702,140.0
AK,Alaska,578,N/A,1016,185,3032,1396,98,81,38,1373,...,281,280,2431,694,98,764,2974,60,1124,240.0
AZ,Arizona,1505,2313,N/A,1135,61262,12372,2726,518,305,7825,...,1490,2136,15342,8736,928,5019,15723,412,11228,1135.0
AR,Arkansas,1103,246,972,N/A,4605,2574,206,35,91,2314,...,75,4749,15287,639,18,1152,1034,88,1514,386.0
CA,California,4004,3986,31489,3276,N/A,20369,10100,1117,4835,23312,...,1190,6214,39343,9146,1634,17578,32640,748,13374,1706.0
CO,Colorado,3113,1763,9771,1318,28742,N/A,3566,193,907,13193,...,1438,3088,25209,3564,1380,6983,6417,480,8586,3646.0
CT,Connecticut,541,257,453,211,4043,763,N/A,385,438,5806,...,137,567,2905,293,466,2136,707,168,221,47.0
DE,Delaware,210,155,127,0,529,152,1074,N/A,251,1116,...,4,157,745,42,240,1420,157,186,118,0.0
DC,District of Columbia,356,123,407,62,3663,680,2162,231,N/A,1101,...,797,1750,237,282,8598,391,260,510,15,NaN
FL,Florida,12391,1990,7312,3876,27384,11850,29220,3056,1498,N/A,...,1191,15782,29051,3620,4372,27785,6952,3313,17044,519.0


In [17]:
df.reset_index(inplace=True)

In [18]:
df = df.rename(columns={'level_0':'abbrev', 'level_1':'state'})

In [19]:
df = df.set_index(['abbrev','state'])

In [20]:
coming_df = df

In [21]:
csv = df.to_csv()

In [22]:
csv

'abbrev,state,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,Georgia,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming\nAL,Alabama,N/A,658,1496,897,3760,1984,1354,228,229,15510,21684,896,64,2609,1981,587,993,2031,2432,144,1009,1312,4342,770,6978,1349,241,345,652,208,1598,527,4984,4061,396,1874,813,382,3668,518,2195,84,9062,7611,748,72,3663,1315,396,1702,140.0\nAK,Alaska,578,N/A,1016,185,3032,1396,98,81,38,1373,994,576,650,844,113,186,793,429,339,232,323,488,1102,1064,117,594,460,184,1270,186,454,478,1438,1555,116,216,476,1285,802,140,381,281,280,2431,694,98,764,2974,60,1124,240.0\nAZ,Arizona,1505,2313,N/A

In [23]:
all_data = []
for i in range(len(states)):
    data = session.execute(f"select sum(movedout) from migration_data_2019 where state1_name = '{states[i]}' group by state1_name, state2_name order by state2_name").all()
    data_fixed = []
    for row in data:
        data_fixed.append(list(map(int, list(row))))
    data_fixed = [data_fixed[i][0] for i in range(len(data_fixed))]
    data_fixed[i] = 0
    all_data.append(data_fixed)

df = pd.DataFrame(data=all_data, index=[states_abbrev, states], columns=df_columns)
df.reset_index(inplace=True)
df = df.rename(columns={'level_0':'abbrev', 'level_1':'state'})
df = df.set_index(['abbrev','state'])
df['total_imm']=coming_df.sum()
df['total_emm']=df.sum(axis=1)
going = df.to_csv()

In [24]:
df.apply(pd.to_numeric)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 51 entries, ('AL', 'Alabama') to ('WY', 'Wyoming')
Data columns (total 53 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Alabama               51 non-null     int64  
 1   Alaska                51 non-null     int64  
 2   Arizona               51 non-null     int64  
 3   Arkansas              51 non-null     int64  
 4   California            51 non-null     int64  
 5   Colorado              51 non-null     int64  
 6   Connecticut           51 non-null     int64  
 7   Delaware              51 non-null     int64  
 8   District of Columbia  51 non-null     int64  
 9   Florida               51 non-null     int64  
 10  Georgia               51 non-null     int64  
 11  Hawaii                51 non-null     int64  
 12  Idaho                 51 non-null     int64  
 13  Illinois              51 non-null     int64  
 14  Indiana               51 non-null     int64  

,,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,total_imm,total_emm
abbrev,state,,,,,,,,,,,,,,,,,,,,,
AL,Alabama,0,578,1505,1103,4004,3113,1082,210,356,12391,...,8168,511,142,2954,997,237,1196,471.0,NaN,105742.0
AK,Alaska,658,0,2313,246,3986,1763,514,155,123,1990,...,3147,1094,284,1148,5034,73,846,271.0,NaN,47302.0
AZ,Arizona,1496,1016,0,972,31489,9771,906,127,407,7312,...,18929,7616,268,2729,11956,159,4816,672.0,NaN,202207.0
AR,Arkansas,897,185,1135,0,3276,1318,422,0,62,3876,...,13548,667,0,926,941,123,1178,252.0,NaN,74493.0
CA,California,3760,3032,61262,4605,0,28742,8086,529,3663,27384,...,73914,18513,2084,18867,49860,773,11208,2315.0,NaN,743502.0
CO,Colorado,1984,1396,12372,2574,20369,0,1526,152,680,11850,...,21111,4681,942,4086,9053,583,5696,4951.0,NaN,214203.0
CT,Connecticut,677,49,1363,103,5050,1783,0,537,1081,14610,...,2703,806,1256,3632,1545,407,665,88.0,NaN,106774.0
DE,Delaware,228,81,518,35,1117,193,770,0,231,3056,...,1400,86,34,1363,274,244,384,59.0,NaN,43159.0
DC,District of Columbia,229,38,305,91,4835,907,876,251,0,929,...,280,434,10194,1192,354,562,7,NaN,NaN,68461.0


In [50]:
coming_df.apply(pd.to_numeric)

,,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
abbrev,state,,,,,,,,,,,,,,,,,,,,,
AL,Alabama,0,658,1496,897,3760,1984,1354,228,229,15510,...,84,9062,7611,748,72,3663,1315,396,1702,140.0
AK,Alaska,578,0,1016,185,3032,1396,98,81,38,1373,...,281,280,2431,694,98,764,2974,60,1124,240.0
AZ,Arizona,1505,2313,0,1135,61262,12372,2726,518,305,7825,...,1490,2136,15342,8736,928,5019,15723,412,11228,1135.0
AR,Arkansas,1103,246,972,0,4605,2574,206,35,91,2314,...,75,4749,15287,639,18,1152,1034,88,1514,386.0
CA,California,4004,3986,31489,3276,0,20369,10100,1117,4835,23312,...,1190,6214,39343,9146,1634,17578,32640,748,13374,1706.0
CO,Colorado,3113,1763,9771,1318,28742,0,3566,193,907,13193,...,1438,3088,25209,3564,1380,6983,6417,480,8586,3646.0
CT,Connecticut,541,257,453,211,4043,763,0,385,438,5806,...,137,567,2905,293,466,2136,707,168,221,47.0
DE,Delaware,210,155,127,0,529,152,1074,0,251,1116,...,4,157,745,42,240,1420,157,186,118,0.0
DC,District of Columbia,356,123,407,62,3663,680,2162,231,0,1101,...,797,1750,237,282,8598,391,260,510,15,NaN


In [51]:
coming_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 51 entries, ('AL', 'Alabama') to ('WY', 'Wyoming')
Data columns (total 51 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Alabama               51 non-null     int64  
 1   Alaska                51 non-null     int64  
 2   Arizona               51 non-null     int64  
 3   Arkansas              51 non-null     int64  
 4   California            51 non-null     int64  
 5   Colorado              51 non-null     int64  
 6   Connecticut           51 non-null     int64  
 7   Delaware              51 non-null     int64  
 8   District of Columbia  51 non-null     int64  
 9   Florida               51 non-null     int64  
 10  Georgia               51 non-null     int64  
 11  Hawaii                51 non-null     int64  
 12  Idaho                 51 non-null     int64  
 13  Illinois              51 non-null     int64  
 14  Indiana               51 non-null     int64  

In [25]:
all_data = []
for i in range(len(states)):
    data = session.execute(f"select sum(movedin) from migration_data_2019 where state1_name = '{states[i]}' group by state1_name, state2_name order by state2_name").all()
    data_fixed = []
    for row in data:
        data_fixed.append(list(map(int, list(row))))
    data_fixed = [data_fixed[i][0] for i in range(len(data_fixed))]
    all_data.append(data_fixed)

df = pd.DataFrame(data=all_data, index=[states_abbrev, states], columns=df_columns)
df.reset_index(inplace=True)
df = df.rename(columns={'level_0':'abbrev', 'level_1':'state'})
df = df.set_index(['abbrev','state'])
coming_df = df
coming_df = coming_df.apply(pd.to_numeric)
for i in range(len(states)):
    df.iloc[i,i] = "N/A"
coming = df.to_csv(path_or_buf="coming.csv")

In [26]:
df

,,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
abbrev,state,,,,,,,,,,,,,,,,,,,,,
AL,Alabama,N/A,658,1496,897,3760,1984,1354,228,229,15510,...,84,9062,7611,748,72,3663,1315,396,1702,140.0
AK,Alaska,578,N/A,1016,185,3032,1396,98,81,38,1373,...,281,280,2431,694,98,764,2974,60,1124,240.0
AZ,Arizona,1505,2313,N/A,1135,61262,12372,2726,518,305,7825,...,1490,2136,15342,8736,928,5019,15723,412,11228,1135.0
AR,Arkansas,1103,246,972,N/A,4605,2574,206,35,91,2314,...,75,4749,15287,639,18,1152,1034,88,1514,386.0
CA,California,4004,3986,31489,3276,N/A,20369,10100,1117,4835,23312,...,1190,6214,39343,9146,1634,17578,32640,748,13374,1706.0
CO,Colorado,3113,1763,9771,1318,28742,N/A,3566,193,907,13193,...,1438,3088,25209,3564,1380,6983,6417,480,8586,3646.0
CT,Connecticut,541,257,453,211,4043,763,N/A,385,438,5806,...,137,567,2905,293,466,2136,707,168,221,47.0
DE,Delaware,210,155,127,0,529,152,1074,N/A,251,1116,...,4,157,745,42,240,1420,157,186,118,0.0
DC,District of Columbia,356,123,407,62,3663,680,2162,231,N/A,1101,...,797,1750,237,282,8598,391,260,510,15,NaN


In [63]:
all_data = []
for i in range(len(states)):
    data = session.execute(f"select sum(movedin) from migration_data_2019 where state1_name = '{states[i]}' group by state1_name, state2_name order by state2_name").all()
    data_fixed = []
    for row in data:
        data_fixed.append(list(map(int, list(row))))
    data_fixed = [data_fixed[i][0] for i in range(len(data_fixed))]
    all_data.append(data_fixed)

df = pd.DataFrame(data=all_data, index=[states_abbrev, states], columns=df_columns)
df.reset_index(inplace=True)
df = df.rename(columns={'level_0':'abbrev', 'level_1':'state'})
df = df.set_index(['abbrev','state'])
coming_df = df
coming_df = coming_df.apply(pd.to_numeric)
for i in range(len(states)):
    coming_df.iloc[i,i] = 0 
for i in range(len(states)):
    df.iloc[i,i] = "N/A"
df.fillna(0, inplace=True)
coming = df.to_csv()

In [78]:
all_data = []
for i in range(len(states)):
    data = session.execute(f"select sum(movedout) from migration_data_2019 where state1_name = '{states[i]}' group by state1_name, state2_name order by state2_name").all()
    data_fixed = []
    for row in data:
        data_fixed.append(list(map(int, list(row))))
    data_fixed = [data_fixed[i][0] for i in range(len(data_fixed))]
    all_data.append(data_fixed)

df = pd.DataFrame(data=all_data, index=[states_abbrev, states], columns=df_columns)
df.reset_index(inplace=True)
df = df.rename(columns={'level_0':'abbrev', 'level_1':'state'})
df = df.set_index(['abbrev','state'])
new_df = df
new_df = new_df.apply(pd.to_numeric)
for i in range(len(states)):
    new_df.iloc[i,i] = 0 
df['total_imm']=coming_df.sum(axis=1)
df['total_emm']=new_df.sum(axis=1)
going_df = df
for i in range(len(states)):
    df.iloc[i,i] = "N/A"
df.fillna(0, inplace=True)
going = df.to_csv()

In [79]:
going_df

,,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,total_imm,total_emm
abbrev,state,,,,,,,,,,,,,,,,,,,,,
AL,Alabama,N/A,578,1505,1103,4004,3113,1082,210,356,12391,...,8168,511,142,2954,997,237,1196,471.0,122552.0,105742.0
AK,Alaska,658,N/A,2313,246,3986,1763,514,155,123,1990,...,3147,1094,284,1148,5034,73,846,271.0,35028.0,47302.0
AZ,Arizona,1496,1016,N/A,972,31489,9771,906,127,407,7312,...,18929,7616,268,2729,11956,159,4816,672.0,299986.0,202207.0
AR,Arkansas,897,185,1135,N/A,3276,1318,422,0,62,3876,...,13548,667,0,926,941,123,1178,252.0,77024.0,74493.0
CA,California,3760,3032,61262,4605,N/A,28742,8086,529,3663,27384,...,73914,18513,2084,18867,49860,773,11208,2315.0,616655.0,743502.0
CO,Colorado,1984,1396,12372,2574,20369,N/A,1526,152,680,11850,...,21111,4681,942,4086,9053,583,5696,4951.0,265031.0,214203.0
CT,Connecticut,677,49,1363,103,5050,1783,N/A,537,1081,14610,...,2703,806,1256,3632,1545,407,665,88.0,81840.0,106774.0
DE,Delaware,228,81,518,35,1117,193,770,N/A,231,3056,...,1400,86,34,1363,274,244,384,59.0,58590.0,43159.0
DC,District of Columbia,229,38,305,91,4835,907,876,251,N/A,929,...,280,434,10194,1192,354,562,7,0,67716.0,68461.0
